In [1]:
### Note new packages that we are importing!!
from cartopy import crs as ccrs, feature as cfeature        ## for plotting geo spatial data
import cmocean                    ## for nice colormaps.
import matplotlib.path as mpath   ## needed for nice looking polar projections.
import momlevel as ml             ## for computing annual mean and seasonal cycle.
import matplotlib.pyplot as plt
import zarr
import xarray as xr
import numpy as np

In [2]:
%matplotlib inline                           
%config InlineBackend.figure_format='retina' 
plt.rcParams['figure.figsize'] = 12,6

### Load in our observations of Sea Ice Concentration from the NOAA OI SST V2 High Resolution Dataset from September 1981 to September 2023 on a 1/4 - degree grid. Here we load in the dataset in as a Zarr store instead of a netcdf file since the dataset is high-resolution (you do not need to know the details of this now). To load in Zarr format data, we use Xarray's `xr.open_zarr()` method instead of `xr.open_dataset`.

### We also load in a file that contains the pre-computed grid cell area for our dataset:



In [3]:
seaice_observations = xr.open_zarr('/home/ees_student/Miranda_Unit5/data/seaice_concentration_data')
seaice_gridcell_area = xr.open_dataset('/home/ees_student/Miranda_Unit5/data/cell_area_seaiceconc.nc')

### Function for creating nice looking NorthPolarStereo projections:

In [4]:
def North_Polar(array,vmin,vmax,cmap):
    subplot_kws = dict(projection=ccrs.NorthPolarStereo()) 
    fig, ax = plt.subplots(figsize=[10, 5],subplot_kw=subplot_kws)
    
    array.plot(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=vmin,
               vmax=vmax,
               extend='both',
               cmap=cmap,levels=30)

    # Limit the map to -50 degrees latitude and below.
    ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
    
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)

    ax.set_boundary(circle, transform=ax.transAxes)
    ax.coastlines()

### Function for creating nice looking SouthPolarStereo projections:

In [5]:
def South_Polar(array,vmin,vmax,cmap):
    subplot_kws = dict(projection=ccrs.SouthPolarStereo()) 
    fig, ax = plt.subplots(figsize=[10, 5],subplot_kw=subplot_kws)
    
    array.plot(ax=ax,
               transform =ccrs.PlateCarree(),
               vmin=vmin,
               vmax=vmax,
               extend='both',
               cmap=cmap, levels=30)

    # Limit the map to -50 degrees latitude and below.
    ax.set_extent([-180, 180, -90, -50], ccrs.PlateCarree())
    
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)

    ax.set_boundary(circle, transform=ax.transAxes)
    ax.coastlines()

### Create a global map using a nice - projection of the time-averaged (Sept 1981 to Sept 2022, full dataset) sea ice concentration. 
* #### Add coastlines to your plot to distinguish land using `ax.coastlines()`.
* #### Make sure to choose appropriate vmin & vmax values for your color bar.
* #### Choose a global projection that is reasonable to display the data .....
  * Explore the Robinson, EckertIV, and other projections.
* #### Choose an appropriate color map to display the data.
  * cmocean maps here: https://matplotlib.org/cmocean/ (hint: what is the data you are displaying??)

### For a full list of projections available via Cartopy please use this reference: https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#platecarree

### Use Xarray's `.groupby()` method and the polar projection function given above to create a polar projection of Northern Hemisphere wintertime-averaged Arctic sea ice averaged over the full time period of observations:

In [6]:
seaice_observations

<xarray.Dataset>
Dimensions:  (time: 505, lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1981-09-01 1981-10-01 ... 2023-09-01
Data variables:
    icec     (time, lat, lon) float32 dask.array<chunksize=(32, 90, 180), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.5
    References:     https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2....
    comment:        Reynolds, et al.(2007) Daily High-Resolution-Blended Anal...
    dataset_title:  NOAA Daily Optimum Interpolation Sea Surface Temperature
    institution:    NOAA/National Centers for Environmental Information
    source:         NOAA/NCEI https://www.ncei.noaa.gov/data/sea-surface-temp...
    title:          NOAA/NCEI 1/4 Degree Daily Optimum Interpolation Sea Surf...
    version:        Version 2.1

In [7]:
NH_SIC = seaice_observations.sel(lat=slice(0,89.875))

In [8]:
NH_SIC

<xarray.Dataset>
Dimensions:  (time: 505, lat: 360, lon: 1440)
Coordinates:
  * lat      (lat) float32 0.125 0.375 0.625 0.875 ... 89.12 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1981-09-01 1981-10-01 ... 2023-09-01
Data variables:
    icec     (time, lat, lon) float32 dask.array<chunksize=(32, 90, 180), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.5
    References:     https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2....
    comment:        Reynolds, et al.(2007) Daily High-Resolution-Blended Anal...
    dataset_title:  NOAA Daily Optimum Interpolation Sea Surface Temperature
    institution:    NOAA/National Centers for Environmental Information
    source:         NOAA/NCEI https://www.ncei.noaa.gov/data/sea-surface-temp...
    title:          NOAA/NCEI 1/4 Degree Daily Optimum Interpolation Sea Surf...
    version:        Version 2.1

In [13]:
NH_SIC_season = NH_SIC.groupby('time.season').mean(dim='time')

In [14]:
NH_SIC_season  

<xarray.Dataset>
Dimensions:  (season: 4, lat: 360, lon: 1440)
Coordinates:
  * lat      (lat) float32 0.125 0.375 0.625 0.875 ... 89.12 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    icec     (season, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.5
    References:     https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2....
    comment:        Reynolds, et al.(2007) Daily High-Resolution-Blended Anal...
    dataset_title:  NOAA Daily Optimum Interpolation Sea Surface Temperature
    institution:    NOAA/National Centers for Environmental Information
    source:         NOAA/NCEI https://www.ncei.noaa.gov/data/sea-surface-temp...
    title:          NOAA/NCEI 1/4 Degree Daily Optimum Interpolation Sea Surf...
    version:        Version 2.1

In [21]:
NH_SIC_winter = NH_SIC_season.sel(season='DJF')

In [17]:
NH_SIC_winter

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 1440)
Coordinates:
  * lat      (lat) float32 0.125 0.375 0.625 0.875 ... 89.12 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
    season   <U3 'DJF'
Data variables:
    icec     (lat, lon) float32 dask.array<chunksize=(90, 180), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.5
    References:     https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2....
    comment:        Reynolds, et al.(2007) Daily High-Resolution-Blended Anal...
    dataset_title:  NOAA Daily Optimum Interpolation Sea Surface Temperature
    institution:    NOAA/National Centers for Environmental Information
    source:         NOAA/NCEI https://www.ncei.noaa.gov/data/sea-surface-temp...
    title:          NOAA/NCEI 1/4 Degree Daily Optimum Interpolation Sea Surf...
    version:        Version 2.1

In [20]:
NH_SIC_winter_annual = ml.util.annual_average(NH_SIC_winter,season='DJF')

TypeError: annual_average() got an unexpected keyword argument 'season'

### Use Xarray's `.groupby()` method and the polar projection function given above to create a polar projection of Northern Hemisphere summertime-averaged Arctic sea ice averaged over the full time period of observations:

### Use Xarray's `.groupby()` method and the polar projection function given above to create a polar projection of Southern Hemisphere wintertime-averaged Arctic sea ice averaged over the full time period of observations:

### Use Xarray's `.groupby()` method and the polar projection function given above to create a polar projection of Southern Hemisphere summertime-averaged Arctic sea ice averaged over the full time period of observations:

## Sea Ice Extent Analysis

#### Total Sea Ice Extent (SIE) in million km^2 is a metric that climate scientists commonly use to monitor sea ice. SIE is computed as:
* The __sum of__ the (fractional sea ice coverage (concentration) * the grid cell area covered by sea ice) = m^2
* Convert m^2 to km^2

#### You have loaded in two datasets that will allow you to do this calculation and reproduce the following plot of the seasonal cycle for Arctic SIE. Produce a plot of Arctic SIE: 
* With the 1982 to 2010 time-average as a thick dashed line
* Use Xarray's `.std()` method to compute the standard deviation (this is just like you did for `.mean(dim='time')`, except your method is now `.std(dim='time')` and place two thin lines on your plot that indicate the 1982 to 2010 time-average + 2 standard deviations and the 1982 to 2010 time-average + 2 standard deviations.
* A thick red line for the annual-cycle of SIE for 2012
* Lines for 2020 and 2021.
* Your x-axis should be in actual months, not integers.

*** Hints: 
* This will involve some slicing in space and time!
* Use Xarray's `.groupby('time.month').mean(dim='time')` method to take your 1982 to 2010 data to an annual / seasonal cycle!
* You can also use .std(dim='time') .....
* Make sure you are checking your DataArrays that you are calculating to make sure you know what you are plotting / the regions / dimensions / etc .... when in doubt, print your array!
* You do not have to do this for this assignment ..... but you can also using ax.fill_between()... specifying your x and y axes just like you did using a Pandas array in our Unit3 module and shade your standard deviations about your mean ....


![](https://eoimages.gsfc.nasa.gov/images/imagerecords/147000/147306/arctic_nsidc_2020259_plot.png)

## Repeat the above excercise but for Antarctic Sea Ice ... 

### Use Xarray's `.groupby()` method and the polar projection function given above to create a polar projection of sea ice concentration for the month when Antarctic sea ice extent is at its maximum ...

### Use Xarray's `.groupby()` method and the polar projection function given above to create a polar projection of sea ice concentration for the month when Antarctic sea ice extent is at its minimum ...

### Use Xarray's `.groupby()` method and the polar projection function given above to create a polar projection of sea ice concentration for the month when Arctic sea ice extent is at its minimum ...

### Use Xarray's `.groupby()` method and the polar projection function given above to create a polar projection of sea ice concentration for the month when Arctic sea ice extent is at its maximum ...